# Processing inputs for the Dynamic Benthic Pelagic Model (DBPM)
**Author:** Denisse Fierro Arcos  
**Date:** 2024-10-10  
  
We will use GFDL-MOM6-COBALT2 (referred to as GFDL from now on) outputs at two horizontal resolutions: 0.25$^{\circ}$ (original grid) and 1$^{\circ}$ (coarsen from original grid) to force the Dynamic Benthic Pelagic Model (DBPM).  

DBPM requires the seven inputs, some of which are directly available as outputs in GFDL, while others need to be calculated from available GFDL outputs:  
- depth (`deptho` in GFDL)  
- small phytoplankton (`phypico-vint` or mole concentration of picoplankton expressed as carbon in sea water in GFDL)  
- large phytoplankton: This variable is not directly available in GFDL, but it can be calculated by substracting small phytoplankton (`phypico-vint` in GFDL) from total phytoplankton (`phyc-vint` or phytoplankton carbon concentration in GFDL)  
- sea surface temperature (`tos` in GFDL)  
- bottom ocean temperature (`tob` in GFDL)  
- detritus (`expc-bot` or downward flux of particulate organic carbon in GFDL)
- export ratio: This variable is estimated from small phytoplankton, large phytoplankton, sea surface temperature and depth using the `getExportRatio` function  

Finally, we will create a spinup using the first 20 years of data available in the `ctrlclim` outputs. We will repeat data for this period six times for a spinup period of 120 years.  
  
All variables will be processed at a global scale and they will be stored in `zarr` format so they can be easily accessed when running DBPM.  

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/dbpm_southern_ocean/scripts/')
import xarray as xr
import pandas as pd
import numpy as np
from glob import glob
from dask.distributed import Client
import useful_functions as uf

## Start a cluster

In [3]:
client = Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44005,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:41761,Total threads: 1
Dashboard: /proxy/46311/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:37033,


## Load relevant GFDL-MOM6-COBALT2 outputs

In [32]:
res = '025deg'
#Base folder where GFDL outputs are stored - obsclim and ctrlclim
gfdl_obsclim = f'/g/data/vf71/fishmip_inputs/ISIMIP3a/global_inputs/obsclim/{res}'
gfdl_ctrlclim = f'/g/data/vf71/fishmip_inputs/ISIMIP3a/global_inputs/ctrlclim/{res}'
#Variables of interest
dbpm_var = ['siconc', 'deptho', 'expc-bot', 'phyc-vint', 'phypico-vint', 'tob', 'tos']
#List of relevant files
gfdl_obs_files = [glob(os.path.join(gfdl_obsclim, f'*_{var}_*'))[0] for var in dbpm_var]
gfdl_ctrl_files = [glob(os.path.join(gfdl_ctrlclim, f'*_{var}_*'))[0] for var in dbpm_var]

## Define location of outputs

In [47]:
gfdl_out = f'/g/data/vf71/la6889/dbpm_inputs/global_gridded/{res}'
os.makedirs(gfdl_out, exist_ok = True)

## Saving `netcdf` files as `zarr` files

### `obsclim`

In [22]:
for f in gfdl_obs_files:
    #Create file path to save outputs
    f_out = os.path.basename(f).replace('.nc', '.zarr')
    f_out = os.path.join(gfdl_out, f_out)
    #Apply function
    uf.netcdf_to_zarr(f, f_out)

/g/data/vf71/la6889/dbpm_southern_ocean/scripts/useful_functions.py:33: FutureWarning: In a future version of xarray to_datetimeindex will default to returning a 'us'-resolution DatetimeIndex instead of a 'ns'-resolution DatetimeIndex. This warning can be silenced by explicitly passing the `time_unit` keyword argument.
  da['time'] = pd.DatetimeIndex(da.indexes['time'].to_datetimeindex(),
/g/data/vf71/la6889/dbpm_southern_ocean/scripts/useful_functions.py:33: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da['time'] = pd.DatetimeIndex(da.indexes['time'].to_datetimeindex(),


### `ctrlclim`

In [36]:
for f in gfdl_ctrl_files:
    #Create file path to save outputs
    f_out = os.path.basename(f).replace('.nc', '.zarr')
    f_out = os.path.join(gfdl_out, f_out)
    #Apply function
    uf.netcdf_to_zarr(f, f_out)

/g/data/vf71/la6889/dbpm_southern_ocean/scripts/useful_functions.py:33: FutureWarning: In a future version of xarray to_datetimeindex will default to returning a 'us'-resolution DatetimeIndex instead of a 'ns'-resolution DatetimeIndex. This warning can be silenced by explicitly passing the `time_unit` keyword argument.
  da['time'] = pd.DatetimeIndex(da.indexes['time'].to_datetimeindex(),
/g/data/vf71/la6889/dbpm_southern_ocean/scripts/useful_functions.py:33: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da['time'] = pd.DatetimeIndex(da.indexes['time'].to_datetimeindex(),


### Area of the grid cell (`areacello`)

In [8]:
if res == '1deg':
    res_am = '60arcmin'
elif res == '025deg':
    res_am = '15arcmin'
area_path = f'/g/data/vf71/shared_resources/grid_cell_area_ESMs/isimip3a/gfdl-mom6-cobalt2_areacello_{res_am}_global_fixed.nc'
file_out = os.path.basename(area_path).replace('.nc', '.zarr')
file_out = os.path.join(gfdl_out, file_out)
uf.netcdf_to_zarr(area_path, file_out)

## Calculating phytoplankton (large and small) and export ratio

In [12]:
#Calculating from ctrlclim data
gfdl_exp = 'ctrlclim'
sphy_ctrl, lphy_ctrl, er_ctrl = uf.getExportRatio(gfdl_out, gfdl_exp)

In [13]:
#Saving datasets
sphy_ctrl.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_sphy_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

lphy_ctrl.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_lphy_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

er_ctrl.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_er_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

In [14]:
#Calculating from obsclim data
gfdl_exp = 'obsclim'
sphy_obs, lphy_obs, er_obs = uf.getExportRatio(gfdl_out, gfdl_exp)

In [16]:
#Saving datasets
sphy_obs.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_sphy_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

lphy_obs.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_lphy_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

er_obs.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_er_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

## Calculate intercept and slope

### `obsclim`

In [18]:
int_obs, slope_obs = uf.GetPPIntSlope(
    glob(os.path.join(gfdl_out, '*obsclim*')))

#Saving datasets
int_obs.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_intercept_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

slope_obs.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_{gfdl_exp}_slope_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

### `ctrlclim`

In [19]:
int_ctrl, slope_ctrl = uf.GetPPIntSlope(
    glob(os.path.join(gfdl_out, '*ctrlclim*')))

#Saving datasets
int_ctrl.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_ctrlclim_intercept_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

slope_ctrl.to_zarr(
    os.path.join(gfdl_out, 
                 f'gfdl-mom6-cobalt2_ctrlclim_slope_{res_am}_global_monthly_1961_2010.zarr'),
          consolidated = True, mode = 'w')

## Creating **spinup** period
We will use data from `ctrlclim` between 1961 and 1980 as our spinup period. Our spinup period will go from 1841 to 1960.

In [41]:
spin_files = [f for f in glob(os.path.join(gfdl_out, '*ctrlclim*')) 
              if 'deptho' not in f]
spinup_period = pd.date_range('1841-01-01', end = '1960-12-31', 
                              freq = 'MS')

In [43]:
#Get start and end year of spinup period to rename file
start_spin = str(spinup_period.year.min())
end_spin = str(spinup_period.year.max())

for f in spin_files:
    f_out = f.replace('ctrlclim', 'spinup').\
    replace('1961', start_spin).replace('2010', end_spin)
    uf.gridded_spinup(f, '1961', '1980', spinup_period, file_out = f_out)

## Creating **stable spinup** period
Based on the data above, we will create a stable spinup period explanding 100 years starting in 1741. Monthly values for this stable spinup period are simply the mean values for the year 1841.

In [48]:
spin_files = [f for f in glob(os.path.join(gfdl_out, '*spinup_*')) 
              if 'deptho' not in f]
spinup_period = pd.date_range('1741-01-01', end = '1840-12-31', 
                              freq = 'MS')

In [50]:
#Get start and end year of spinup period to rename file
start_spin = str(spinup_period.year.min())
end_spin = str(spinup_period.year.max())

for f in spin_files:
    f_out = f.replace('spinup', 'stable-spin').\
    replace('1841', start_spin).replace('1960', end_spin)
    uf.gridded_spinup(f, '1841-01', '1841-12', spinup_period, mean_spinup = True,
                      file_out = f_out)

## Creating sea ice masks
Sea ice masks identify grid cells that are not accessible to fishers when spreading fishing effort. Any grid cells with a sea ice concentration (SIC) of 15% or more will be set to `NA`, therefore inacessible to be harvested.

In [ ]:
#Identifying sea ice files
si_files = glob(f'/g/data/vf71/la6889/dbpm_inputs/global_gridded/{res}/*siconc*')

# Create masks for all sea ice files
for f in si_files:
    #Load file
    da = xr.open_zarr(f)['siconc']
    #Identify grid cells with at least 15% SIC and add up grid cells along latitude
    da_mask = xr.where(da >= 15, True, False)
    #Rechunk data
    da_mask = da_mask.chunk({'time': '500MB'})
    #Update data array variable name
    da_mask.name = 'simask'
    #Create file path to save outputs
    f_out = f.replace('siconc', 'simask')
    #Save result 
    da_mask.to_zarr(f_out, consolidated = True, mode = 'w')